In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision import models, transforms, datasets

import torch.optim as optim
from torch.utils.data import DataLoader

import PIL.Image as Image
from tqdm import tqdm
import os
import time

from sklearn.metrics import f1_score

from model.vgg_16 import *
from model.hybrid_CNN import *


In [12]:
# hyper parameters
num_epochs = 1
num_classes = 2
batch_size = 1 # WE WANT IMAGES TO PASS HYBRID CONV LAYER ONE BY ONE
learning_rate = 0.001
model_name = vgg16_bn(pretrained=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
model_name

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [14]:
# get dataset
def load_data(batch_size):
    """
    return the train/val/test dataloader
    """
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)
    ])
    
    train_dataset = datasets.CelebA(root='./data',
                                    split='train',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)
    val_dataset = datasets.CelebA(root='./data',
                                    split='valid',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)
    test_dataset = datasets.CelebA(root='./data',
                                    split='test',
                                    target_type='attr',
                                    transform=transform,
                                    download=False)

    # data loader
    train_loader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True)
    val_loader = DataLoader(dataset=val_dataset,
                                batch_size=batch_size,
                                shuffle=False)
    test_loader = DataLoader(dataset=test_dataset,
                                batch_size=batch_size,
                                shuffle=False)
    
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_data(batch_size)

In [15]:
def initialize_model(model_name, learning_rate, num_classes, device):
    """
    initialize the model (pretrained vgg16_bn)
    define loss function and optimizer and move data to gpu if available
    
    return:
        model, loss function(criterion), optimizer
    """
    model = model_name
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    
    model = model_name.to(device)
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    return model, criterion, optimizer


model, criterion, optimizer = initialize_model(model_name, learning_rate, num_classes, device)

In [6]:
def make_plots(step_hist, loss_hist, epoch=0):
    plt.plot(step_hist, loss_hist)
    plt.xlabel('train_iterations')
    plt.ylabel('Loss')
    plt.title('epoch'+str(epoch+1))
    plt.savefig('epoch_1')
    plt.clf()

In [7]:
def train(train_loader, model, criterion, optimizer, num_epochs, device):
    """
    Move data to GPU memory and train for specified number of epochs
    Also plot the loss function and save it in `Figures/`
    Trained model is saved as `cnn.ckpt`
    """
    for epoch in range(num_epochs): # repeat the entire training `num_epochs` times
        # for each training sample
        loss_hist = []
        step_hist = []
        for i, (images, labels) in tqdm(enumerate(train_loader)):
            # move to gpu if available
            label = labels[:, 2]   # attractiveness label
            cov_attr = labels[:, 20]    # gender (male/female)   
            cov_attr = (cov_attr + 1) // 2  # map from {-1, 1} to {0, 1}
            
            images = images.to(device)
            label = label.to(device)
            
            # forward pass
            outputs = model(images, cov_attr)    # model takes covariate here
            loss = criterion(outputs, label) 
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # print
            if (i+1) % 100 == 0:
                print('Epoch: [{}/{}], Step[{}/{}], Loss:{:.4f}' \
                    .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
                loss_hist.append(loss.item())
                step_hist.append(i+1)
        
        make_plots(step_hist, loss_hist, epoch)
        
    torch.save(model.state_dict(), 'cnn2.ckpt')

In [8]:
# train model
train(train_loader, model, criterion, optimizer, num_epochs, device)

0it [00:01, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 2.45 GiB already allocated; 269.16 MiB free; 2.52 GiB reserved in total by PyTorch)

In [9]:
def evaluate(val_loader, model, device):
    """
    Run the validation set on the trained model
    """
    
    model_path = "model/vgg16_bn_32.ckpt"
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    
    model.eval() # BatchNorm uses moving mean/variance instead of mini-batch mean/variance
    with torch.no_grad():
        # initialize the stats
        correct = 0
        total = 0
        y_true = []
        y_pred = []
        # pass through testing data once
        for images, labels in val_loader:

            label = labels[:, 2]
#             cov_attr = labels[:, 20]    # gender (male/female)   
#             cov_attr = (cov_attr + 1) // 2  # map from {-1, 1} to {0, 1}
            # again move to device first
            images = images.to(device)
            label = label.to(device)
            
            # forward once
            outputs = model(images)
            
            # instead of calculating loss we will get predictions
            # it's essetially outputs just reformatting imo
            _, predicted = torch.max(outputs.data, 1)
            
            # accumulate stats
            y_true.append(label)
            y_pred.append(predicted)
            total += label.size(0) # yeah again, number of elements in the tensor
            correct += (label == predicted).sum().item()

        # print
        print('Test accuracy on validation set: {}%' \
                .format(100 * correct / total))
        
# run validation set
evaluate(val_loader, model, device)

KeyboardInterrupt: 

In [42]:
model_path = "model/vgg16_bn_32.ckpt"
state_dict = torch.load(model_path)
model.load_state_dict(state_dict)

model.eval() 

with torch.no_grad():
    # initialize the stats
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    # pass through testing data once
    for images, labels in tqdm(val_loader):

        label = labels[:, 2]
        # cov_attr = labels[:, 20]   
        # cov_attr = (cov_attr + 1) // 2  # (32,)
        # again move to device first
        
        images = images.to(device) # (1, 3, 224, 224) -> (32, 3, 224, 224)
        label = label.to(device)   # (1,) -> (32,)

        # forward once
        outputs = model(images)
        # outputs = model(images, cov_attr)  # forward has to take a batch

        # instead of calculating loss we will get predictions
        # it's essetially outputs just reformatting imo
        _, predicted = torch.max(outputs.data, 1)

        # accumulate stats
        y_true.append(label.item())
        y_pred.append(predicted.item())
        total += label.size(0) # yeah again, number of elements in the tensor
        correct += (label == predicted).sum().item()

    # print
    print('Test accuracy on validation set: {}%' \
            .format(100 * correct / total))

100%|████████████████████████████████████████████████████████████████████████████| 19867/19867 [24:10<00:00, 13.70it/s]

Test accuracy on validation set: 80.66139829868627%


In [46]:
from sklearn.metrics import f1_score

In [47]:
f1_score(y_true, y_pred)

0.8142346001353835

In [48]:
y_true = torch.as_tensor(y_true)
y_pred = torch.as_tensor(y_pred)

f1_score(y_true, y_pred)

0.8142346001353835

In [52]:
a = [[1, 0],
     [1, 0]]

b = [[1, 1],
     [1, 1]]

f1_score(a, b, average='micro')

0.6666666666666666

In [54]:
torch.as_tensor([1, 2, 3])

tensor([1, 2, 3])

In [ ]:
li